In [2]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [16]:
label =[[  461,     8, 10433,     7,    13,     8, 15481,     6,    34,    19,
           614,    12,  3034,    24,    16,   325,  3767,    26,  7902,    54,
          1590, 15121,  1364,   145,    24,     5,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [  148,   410,    59,   580,   140,   893,     5,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]]
label = torch.Tensor(label).long().cuda()
pred = [[ 1363,  1661,    18,    77,    18, 22098,  3527,  3527,  3527,  3527,
          3527,    30,  9682,  9682,  9682,  9682,  9682,  9682,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606,   606,   606,   606,   606,   606,
           606,   606,   606,   606,   606],
        [   16,     8,  1611,  3545,  3545, 11729, 11729,     7,    21,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,  2958,
          2958,  2958,  2958,  2958,  2958]]
pred = torch.Tensor(pred).long().cuda()

In [17]:
model = torch.load('./model/D_B.pt').train()

In [5]:

label = en(['I have some sympathy with the last speaker: sometimes the Committee on Budgetary Control gets so involved with the important work they are doing, that they are unable to see the wood for the trees.', 'I also welcome the presentation on behalf of the Legal Affairs Committee and the Economic and Monetary Affairs Committee which took a balanced approach to the major and indeed legitimate concerns of the Committee on Budgetary Control about fraud.'])[0]
# [           'I agree with the previous speaker: sometimes the Committee on Budgetary Control is so deep in its important work that it does not see the forest behind trees.', 'I also welcome the statements made on behalf of the Committee on Legal Affairs and the Internal Market and the Committee on Economic and Monetary Affairs, which took a fairly balanced position on the serious and quite justified concerns of the Committee on Budgetary Control with regard to fraud.']

pred = en([ 'I agree with the previous speaker: sometimes the Committee on Budgetary Control is so deep in its important work that it does not see the forest behind trees.', 'I also welcome the statements made on behalf of the Committee on Legal Affairs and the Internal Market and the Committee on Economic and Monetary Affairs, which have taken a fairly balanced position on the serious and quite justified concerns of the Committee on Budgetary Control with regard to fraud.'])[0]

In [18]:
#real sentences
x = label.cuda()
#delte 0 give huge diff


# x = torch.Tensor(x).long().cuda()
print(x)
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_label =  model.classifier(distr)#(bs,1)
print(ret_label)

print(model(x,x_attn))

tensor([[  461,     8, 10433,     7,    13,     8, 15481,     6,    34,    19,
           614,    12,  3034,    24,    16,   325,  3767,    26,  7902,    54,
          1590, 15121,  1364,   145,    24,     5,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [12]:
#real sentences
x = pred.cuda()

# x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
print(torch.sum(distr,1).shape)
print(torch.sum(x_attn,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_pred =  model.classifier(distr)#(bs,1)
print(ret_pred)


print(model(x,x_attn))

torch.Size([2, 255])
		---------embedding
torch.Size([2, 255, 512])
tensor([[[-15.8953,   0.3812,   3.2571,  ...,  -4.6822,  27.4347,  17.6696],
         [ -1.9772,  -6.5443,  28.4213,  ..., -56.8083, -72.0014,   8.4897],
         [ 17.9072,   5.1582,   7.1158,  ...,  17.9551,  -6.4665,  -2.5419],
         ...,
         [ 11.3657,  -8.1172,  13.6794,  ..., -25.2435,   3.1112,  -0.7454],
         [ 11.3657,  -8.1172,  13.6794,  ..., -25.2435,   3.1112,  -0.7454],
         [ 11.3657,  -8.1172,  13.6794,  ..., -25.2435,   3.1112,  -0.7454]],

        [[ -5.6621,   1.9777,  -0.9974,  ...,  -8.8201,   4.2981,   0.9969],
         [ 12.2070,   6.7962,  10.2948,  ...,  11.7214,   6.2806,  45.4166],
         [ 16.8151, -12.1546,  30.7473,  ...,   1.9031,  -8.6687,  -0.9308],
         ...,
         [ -2.4141,  -7.9422,  11.3802,  ..., -25.6851,  13.1921,  13.8277],
         [ -2.4141,  -7.9422,  11.3802,  ..., -25.6851,  13.1921,  13.8277],
         [ -2.4141,  -7.9422,  11.3802,  ..., -25.6851,

In [8]:
criterionGAN = torch.nn.MSELoss()
pred_real = ret_label
print(pred_real)
loss_D_real = criterionGAN(pred_real, torch.ones((pred_real.shape[0],1),device='cuda'))
print(loss_D_real)
# Fake
pred_fake = ret_pred
print(pred_real)
loss_D_fake = criterionGAN(pred_fake, torch.zeros((pred_fake.shape[0],1),device='cuda'))
# Combined loss and calculate gradients
print(loss_D_fake)


loss_D = (loss_D_real + loss_D_fake) * 0.5
print(loss_D)

tensor([[0.9922],
        [0.9853]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.9425, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.4713, device='cuda:0', grad_fn=<MulBackward0>)


,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier

## FAKE dont end with 1, cuz i use model.generate but not self.generate